In [2]:
from arff2pandas import a2p
import pandas as pd
import numpy as np


Treinando ensembles para a base JM1

In [3]:
with open('jm1.arff') as f:
    df = a2p.load(f)
    print(df)

       loc@NUMERIC  v(g)@NUMERIC  ev(g)@NUMERIC  iv(g)@NUMERIC  n@NUMERIC  \
0              1.1           1.4            1.4            1.4        1.3   
1              1.0           1.0            1.0            1.0        1.0   
2             72.0           7.0            1.0            6.0      198.0   
3            190.0           3.0            1.0            3.0      600.0   
4             37.0           4.0            1.0            4.0      126.0   
5             31.0           2.0            1.0            2.0      111.0   
6             78.0           9.0            5.0            4.0        0.0   
7              8.0           1.0            1.0            1.0       16.0   
8             24.0           2.0            1.0            2.0        0.0   
9            143.0          22.0           20.0           10.0        0.0   
10            73.0          10.0            4.0            6.0        0.0   
11            83.0          11.0           10.0            7.0        0.0   

Base com missing values, preencher com a media -> 4 valores nan

In [4]:
df = df.interpolate()

In [5]:
input_features = df.drop(["defects@{false,true}"], axis=1)

Base desbalanceada -> lembrar de pegar k-fold estratificado

In [15]:
output_class = df["defects@{false,true}"]
len(output_class[output_class == "true"])

2106

In [25]:
len(output_class[output_class == "false"])

8779

In [16]:
output_class = np.where(output_class == 'true', 1, 0)


In [107]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import accuracy, roc_auc_score, f1_score, 
# skf = StratifiedKFold(n_splits=10)
# cv_results = []
# cv_results_proba = []
# for train_index, test_index in skf.split(input_features, output_class):
#     X_train, y_train = SMOTE().fit_sample(input_features.as_matrix()[train_index], output_class[train_index])
#     X_test, y_test = input_features.as_matrix()[test_index], output_class[test_index]
    
#     result = []
#     result_proba = []    
#     for name, clf in [("Decision Tree", DecisionTreeClassifier()), ("Perceptron", Perceptron(max_iter=1000))]:
#         print(name)
#         for perc in [0.5]:
#             es = BaggingClassifier(base_estimator=clf, n_estimators=100, 
#                            max_samples=perc, max_features=1.0, bootstrap=True, 
#                            bootstrap_features=False, n_jobs=4)
            
#             es.fit(X_train, y_train)
#             result.append(es.predict(X_test))
#             result_proba.append(es.predict_proba(X_test))

#             scores = cross_validate(es, input_features, output_class, scoring=scoring,
#                              cv=10, return_train_score=False)
            
#             scores.pop('fit_time', None)
#             scores.pop('score_time', None)

#             if(start):          
#                 start = False


#                 for key in scores.keys():
#                     print(key, end='\t')
#                 print()

#             print(perc, end='\t')
#             for val in scores.values():
#                 print("{}+-({})".format(str(round(np.mean(val), 2)), str(round(np.std(val), 2))), end='\t')
#             print()
            

# for train, test in skf.split(input_features, output_class):
#     #train, test = SMOTE().fit_sample(np.array(train), np.array(test).reshape(-1, 1))
#     print("%s %s" % (train, test))
#     #cv_balanced_folds.append((train,test))

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

In [19]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer

#Taxa de negativos verdadeiros
def specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)
    return specificity

#Tava de positivos verdadeiros
def sensitivity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp+fn)
    return sensitivity

#Media geometrica entre as taxas
def gmean(y_true, y_pred):
    import math
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp+fn)
    return (specificity*sensitivity)**(1/2)



In [71]:
def get_bagging_results(input_features, output_class):
    import warnings
    warnings.filterwarnings("ignore")

    scoring = {'acc': 'accuracy',
               'auc': 'roc_auc',
               'specificity': make_scorer(specificity),
               'sensitivity': make_scorer(sensitivity),
               'gmean': make_scorer(gmean),
               'precision': 'precision',
               'recall': 'recall',
               'f1': 'f1'

    }

    print("----Bagging----")
    start = True
    for name, clf in [("Decision Tree", DecisionTreeClassifier()), ("Perceptron", Perceptron(max_iter=1000))]:
        print(name)
        for perc in [0.5, 0.6, 0.7, 0.8, 0.8, 1.0]:
            es = BaggingClassifier(base_estimator=clf, n_estimators=100, 
                           max_samples=perc, max_features=1.0, bootstrap=True, 
                           bootstrap_features=False, n_jobs=4)

            scores = cross_validate(es, input_features, output_class, scoring=scoring,
                             cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=45), return_train_score=False)
            
            scores.pop('fit_time', None)
            scores.pop('score_time', None)

            if(start):          
                start = False


                for key in scores.keys():
                    print(key, end='\t')
                print()
            
            for val in scores.values():
                print("{}+-({})".format(str(round(np.mean(val), 2)), str(round(np.std(val), 2))), end='\t')
            print()
            

In [72]:
get_bagging_results(input_features, output_class)

----Bagging----
Decision Tree
test_acc	test_auc	test_specificity	test_sensitivity	test_gmean	test_precision	test_recall	test_f1	
0.82+-(0.0)	0.75+-(0.01)	0.96+-(0.0)	0.21+-(0.02)	0.45+-(0.02)	0.57+-(0.03)	0.21+-(0.02)	0.31+-(0.02)	
0.82+-(0.0)	0.76+-(0.02)	0.96+-(0.0)	0.22+-(0.02)	0.46+-(0.02)	0.56+-(0.02)	0.22+-(0.02)	0.32+-(0.02)	
0.82+-(0.01)	0.75+-(0.02)	0.96+-(0.0)	0.24+-(0.01)	0.48+-(0.02)	0.56+-(0.03)	0.24+-(0.01)	0.33+-(0.02)	
0.82+-(0.0)	0.75+-(0.02)	0.96+-(0.0)	0.24+-(0.02)	0.48+-(0.02)	0.56+-(0.02)	0.24+-(0.02)	0.34+-(0.02)	
0.82+-(0.0)	0.75+-(0.02)	0.96+-(0.01)	0.24+-(0.02)	0.48+-(0.02)	0.57+-(0.03)	0.24+-(0.02)	0.34+-(0.02)	
0.81+-(0.0)	0.75+-(0.02)	0.95+-(0.0)	0.24+-(0.02)	0.48+-(0.02)	0.54+-(0.02)	0.24+-(0.02)	0.33+-(0.02)	
Perceptron
0.79+-(0.01)	0.63+-(0.02)	0.94+-(0.01)	0.13+-(0.03)	0.35+-(0.04)	0.36+-(0.05)	0.13+-(0.03)	0.2+-(0.03)	


KeyboardInterrupt: 

In [ ]:
def get_random_subspace_results(input_features, output_class):
    import warnings
    warnings.filterwarnings("ignore")

    scoring = {'acc': 'accuracy',
               'auc': 'roc_auc',
               'specificity': make_scorer(specificity),
               'sensitivity': make_scorer(sensitivity),
               'gmean': make_scorer(gmean),
               'precision': 'precision',
               'recall': 'recall',
               'f1': 'f1'

    }
    
    print("----Random Subspace----")
    start = True
    for name, clf in [("Decision Tree", DecisionTreeClassifier()), ("Perceptron", Perceptron(max_iter=1000))]:
        print(name)
        es = BaggingClassifier(base_estimator=clf, n_estimators=100, 
                       max_samples=1.0, max_features=0.5, bootstrap=False, 
                       bootstrap_features=True, n_jobs=2)

        scores = cross_validate(es, input_features, output_class, scoring=scoring,
                         cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=45), return_train_score=False)
        
        scores.pop('fit_time', None)
        scores.pop('score_time', None)
        
        if(start):          
            start = False
            for key in scores.keys():
                print(key, end='\t')
            print()
            
        for val in scores.values():
            print("{}+-({})".format(str(round(np.mean(val), 2)), str(round(np.std(val), 2))), end='\t')
        print()

In [ ]:
get_random_subspace_results(input_features, output_class)

Treinando ensembles para a base CM1

In [46]:
with open('cm1.arff') as f:
    cm1 = a2p.load(f)
    print(cm1)

     loc@NUMERIC  v(g)@NUMERIC  ev(g)@NUMERIC  iv(g)@NUMERIC  n@NUMERIC  \
0            1.1           1.4            1.4            1.4        1.3   
1            1.0           1.0            1.0            1.0        1.0   
2           24.0           5.0            1.0            3.0       63.0   
3           20.0           4.0            4.0            2.0       47.0   
4           24.0           6.0            6.0            2.0       72.0   
5           24.0           6.0            6.0            2.0       72.0   
6            7.0           1.0            1.0            1.0       11.0   
7           12.0           2.0            1.0            2.0       23.0   
8           25.0           5.0            5.0            5.0      107.0   
9           46.0          15.0            3.0            1.0      239.0   
10          34.0           5.0            5.0            1.0      155.0   
11          10.0           2.0            1.0            1.0       35.0   
12          23.0         

In [47]:
cm1_input = cm1.drop(["defects@{false,true}"], axis=1)
cm1_output = cm1["defects@{false,true}"]

In [8]:
len(cm1_output[cm1_output == "true"])

49

In [9]:
len(cm1_output[cm1_output == "false"])

449

In [48]:
cm1_output = np.where(cm1_output == 'true', 1, 0)

In [69]:
get_bagging_results(cm1_input, cm1_output)

----Bagging----
Decision Tree
test_acc	test_auc	test_specificity	test_sensitivity	test_gmean	test_precision	test_recall	test_f1	
0.88+-(0.03)	0.76+-(0.1)	0.97+-(0.03)	0.06+-(0.1)	0.14+-(0.21)	0.23+-(0.4)	0.06+-(0.1)	0.1+-(0.15)	


In [70]:
get_random_subspace_results(cm1_input, cm1_output)

----Random Subspace----
Decision Tree
test_acc	test_auc	test_specificity	test_sensitivity	test_gmean	test_precision	test_recall	test_f1	
0.89+-(0.04)	0.74+-(0.09)	0.97+-(0.03)	0.08+-(0.11)	0.18+-(0.22)	0.28+-(0.39)	0.08+-(0.11)	0.12+-(0.16)	
